# Fetch IRSA Dust E(B-V)

When running BADASS for multiple spectra using the multiprocessing notebook, BADASS will hangup when trying to fit spectra it has not previously obtained E(B-V) values for via Astroquery's [IRSA Dust Extinction Service Query](https://astroquery.readthedocs.io/en/latest/irsa/irsa_dust.html).  This is a known issue (see [this](https://github.com/astropy/astroquery/issues/684).  The problem stems from the fact that `IrsaDust. get_query_table()` treats multiple Python subprocesses as a single-process. For example, if you are running 4 subprocesses (fitting 4 spectra simultaneously), it will only query the last process of the four, and leave the first three hanging.  

Luckily there is a workaround.  `IrsaDust. get_query_table()` stores previous queries on your local machine so they can be accessed without looking them up every single time.  The solution is to simply query E(B-V) values for all of your objects before fitting, which seems dumb but it's the only workaround and its quick.   

In [ ]:
import numpy as np
from astropy.io import fits
from astroquery.irsa_dust import IrsaDust
import astropy.units as u
from astropy import coordinates
import glob
import natsort

In [ ]:
def get_dust(spec_fold):
    # Get path of of spectra file
    spec_file = glob.glob(spec_fold+'/*.fits')[0]
    # Get object name 
    obj_name = spec_fold.split('/')[-1]
    try:
        # Load the data
        hdu = fits.open(spec_file)
        specobj = hdu[2].data
        try:
            ra  = hdu[0].header['RA']
            dec = hdu[0].header['DEC']
        except:
            ra  = specobj['PLUG_RA'][0]
            dec = specobj['PLUG_DEC'][0]

        co = coordinates.SkyCoord(ra=ra, dec=dec, unit=(u.deg, u.deg), frame='fk5')
        table = IrsaDust.get_query_table(co,section='ebv')
        ebv = table['ext SandF mean'][0]

        hdu.close()
        print(' %s,  %0.3f' % (obj_name, ebv))
        return ebv
    except: 
        print('\n %s has no (RA,DEC) in header! \n' % obj_name)
        return None

### Directories


In [ ]:
spec_dir  = '../examples/'
# spec_dir  = '/Users/rem/paper_3_nls1_m_sigma/unfit_spectra_ALL/'

spec_fold = natsort.natsorted(glob.glob(spec_dir+'*'))
print len(spec_fold)

### Run

In [ ]:
for i in range(len(spec_fold)):
    ebv = get_dust(spec_fold[i])
    
print('\n Done')